# **\[PyG\]** lesson6: GCN

신록예찬  
2023-07-19

# ref

Kipf and Welling (2016): <https://arxiv.org/abs/1609.02907>

# layer-wise progagation rule

Kipf and Welling (2016) use

$$H^{(l+1)} = \sigma\big(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}\big)$$

Kipf, Thomas N, and Max Welling. 2016. “Semi-Supervised Classification
with Graph Convolutional Networks.” *arXiv Preprint arXiv:1609.02907*.